In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from swiss_rounds_simulator.utils import simulate_tournament, simulate_n_tournaments, compare_settings
from swiss_rounds_simulator.utils import  initiate_league, assign_opponents, play_round


New LDC format places of interest :
* The 8 first teams qualify directly to round of 16
* From 9 to 24 : pseudo round of 32
* Interesting teams : i (9th) and y(25th)  

Format :
* 36 teams,
* 8 games,
* Hat determination of the first 2 games

LoL :
* 16 teams,
* Top 8 to Quarter finales,
* 6 games (not exactly, teams with 3 win qualifies, team with 3 loss gets eliminated)


In [2]:
n_teams = 36
n_rounds = 8

lt = simulate_tournament(nb_teams=n_teams, nb_games=n_rounds, method='probabilistic', delta_level='linear', verbose = False, strategies={'i':[1]})
for i in range(n_rounds+1):
    sub_lt = lt.query(f"Nb_win == {n_rounds-i}")
    if sub_lt.shape[0] != 0:
        display(sub_lt)

,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
g,6,0.833333,[],8,0,1.0,0.59,ai,Win,k,...,Win,f,Win,h,Win,m,Win,l,Win,1


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
t,19,0.472222,[],7,1,0.88,0.66,af,Win,a,...,Loss,j,Win,s,Win,h,Win,d,Win,2


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
h,7,0.805556,[],6,2,0.75,0.69,aa,Win,r,...,Win,s,Win,g,Loss,t,Loss,j,Win,3
m,12,0.666667,[],6,2,0.75,0.59,n,Win,ae,...,Loss,y,Win,i,Win,g,Loss,f,Win,4


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
s,18,0.5,[],5,3,0.62,0.59,o,Win,j,...,Win,h,Loss,t,Loss,c,Loss,w,Win,5
a,0,1.0,[],5,3,0.62,0.50,u,Win,t,...,Loss,p,Win,n,Loss,w,Win,aa,Win,6
j,9,0.75,[],5,3,0.62,0.56,c,Win,s,...,Win,t,Loss,w,Win,i,Win,h,Loss,7
f,5,0.861111,[],5,3,0.62,0.56,aj,Win,ad,...,Win,g,Loss,l,Win,q,Win,m,Loss,7
b,1,0.972222,[],5,3,0.62,0.55,v,Win,p,...,Loss,aa,Loss,ae,Win,y,Win,q,Win,9
k,10,0.722222,[],5,3,0.62,0.52,l,Win,g,...,Loss,ae,Win,aa,Loss,ab,Win,v,Win,10
d,3,0.916667,[],5,3,0.62,0.47,ah,Loss,ac,...,Win,e,Win,y,Win,aa,Win,t,Loss,11
l,11,0.694444,[],5,3,0.62,0.61,k,Loss,aa,...,Win,q,Win,f,Loss,n,Win,g,Loss,12
i,8,0.777778,[1],5,3,0.62,0.48,ae,Loss,e,...,Win,o,Win,m,Loss,j,Loss,n,Win,13
p,15,0.583333,[],5,3,0.62,0.48,e,Win,b,...,Win,a,Loss,x,Win,o,Win,c,Win,13


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
q,16,0.555556,[],4,4,0.5,0.56,w,Loss,v,...,Win,l,Loss,r,Win,f,Loss,b,Loss,15
n,13,0.638889,[],4,4,0.5,0.52,m,Loss,ai,...,Loss,ad,Win,a,Win,l,Loss,i,Loss,16
ab,27,0.25,[],4,4,0.5,0.50,ac,Win,y,...,Loss,w,Loss,u,Win,k,Loss,e,Win,17
aa,26,0.277778,[],4,4,0.5,0.58,h,Loss,l,...,Win,b,Win,k,Win,d,Loss,a,Loss,18
v,21,0.416667,[],4,4,0.5,0.50,b,Loss,q,...,Win,af,Win,o,Win,r,Win,k,Loss,19
y,24,0.333333,[],4,4,0.5,0.50,x,Win,ab,...,Win,m,Loss,d,Loss,b,Loss,u,Win,19
r,17,0.527778,[],4,4,0.5,0.50,ag,Win,h,...,Loss,c,Win,q,Loss,v,Loss,af,Win,21
c,2,0.944444,[],4,4,0.5,0.50,j,Loss,af,...,Win,r,Loss,e,Win,s,Win,p,Loss,22
x,23,0.361111,[],4,4,0.5,0.41,y,Loss,ag,...,Loss,z,Win,p,Loss,ac,Win,ae,Win,23


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
af,31,0.138889,[],3,5,0.38,0.47,t,Loss,c,...,Loss,v,Loss,ah,Win,ae,Win,r,Loss,24
u,20,0.444444,[],3,5,0.38,0.45,a,Loss,o,...,Loss,ac,Win,ab,Loss,ad,Win,y,Loss,25
w,22,0.388889,[],3,5,0.38,0.56,q,Win,ah,...,Loss,ab,Win,j,Loss,a,Loss,s,Loss,26
e,4,0.888889,[],3,5,0.38,0.44,p,Loss,i,...,Win,d,Loss,c,Loss,z,Win,ab,Loss,27
ac,28,0.222222,[],3,5,0.38,0.42,ab,Loss,d,...,Win,u,Loss,ag,Win,x,Loss,z,Win,28
o,14,0.611111,[],3,5,0.38,0.45,s,Loss,u,...,Win,i,Loss,v,Loss,p,Loss,ad,Win,29
ag,32,0.111111,[],3,5,0.38,0.34,r,Loss,x,...,Loss,ah,Win,ac,Loss,ai,Win,aj,Win,30


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
ad,29,0.194444,[],2,6,0.25,0.44,z,Win,f,...,Loss,n,Loss,aj,Win,u,Loss,o,Loss,31
ae,30,0.166667,[],2,6,0.25,0.52,i,Win,m,...,Win,k,Loss,b,Loss,af,Loss,x,Loss,32
z,25,0.305556,[],2,6,0.25,0.39,ad,Loss,aj,...,Loss,x,Loss,ai,Win,e,Loss,ac,Loss,33
aj,35,0.027778,[],2,6,0.25,0.31,f,Loss,z,...,Loss,ai,Win,ad,Loss,ah,Win,ag,Loss,34


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opp,R1_res,R2_opp,...,R4_res,R5_opp,R5_res,R6_opp,R6_res,R7_opp,R7_res,R8_opp,R8_res,Rank
ai,34,0.055556,[],1,7,0.12,0.39,g,Loss,n,...,Loss,aj,Loss,z,Loss,ag,Loss,ah,Win,35
ah,33,0.083333,[],1,7,0.12,0.39,d,Win,w,...,Loss,ag,Loss,af,Loss,aj,Loss,ai,Loss,36


In [3]:
simu = simulate_n_tournaments(n_tournaments=10,nb_teams=n_teams, nb_games= n_rounds, delta_level='linear')
simu

,Level,Strategy,Avg_WR,Avg_Rank
b,0.97,[],0.660,8.9
h,0.81,[],0.637,9.4
i,0.78,[],0.635,9.4
c,0.94,[],0.637,9.5
g,0.83,[],0.648,10.4
a,1.00,[],0.676,10.9
d,0.92,[],0.650,11.0
l,0.69,[],0.612,11.1
j,0.75,[],0.598,11.6
e,0.89,[],0.624,12.1


In [13]:
comp = True
if comp :
    compare_settings(n_tournaments=4, 
                 n_teams=18, 
                 n_rounds = 6, 
                 delta_level ='linear',
                 strategies={'m':[1]}, 
                 probabilistic=True, 
                 deterministic=True)


Probabilistic resolution


,Level,Strategy,Strategic_avg_WR,Strategic_avg_Rank,Control_avg_WR,Control_avg_Rank,Delta_WR,Delta_Rank
c,0.89,[],0.75,3.75,0.54,7.00,0.21,-3.25
a,1.00,[],0.75,3.75,0.58,7.25,0.16,-3.50
h,0.61,[],0.66,4.75,0.46,11.25,0.21,-6.50
d,0.83,[],0.67,5.50,0.75,2.50,-0.08,3.00
f,0.72,[],0.58,5.75,0.75,3.50,-0.17,2.25
b,0.94,[],0.54,7.50,0.79,3.25,-0.25,4.25
i,0.56,[],0.58,8.50,0.42,11.75,0.17,-3.25
l,0.39,[],0.50,8.75,0.58,7.25,-0.08,1.50
g,0.67,[],0.50,9.00,0.71,3.50,-0.21,5.50
e,0.78,[],0.54,9.25,0.58,7.50,-0.04,1.75


---------------------------------------------------
Deterministic resolution


,Level,Strategy,Strategic_avg_WR,Strategic_avg_Rank,Control_avg_WR,Control_avg_Rank,Delta_WR,Delta_Rank
a,1.00,[],1.00,1.00,1.00,1.00,0.00,0.00
b,0.94,[],0.83,2.25,0.83,2.25,0.00,0.00
c,0.89,[],0.71,3.25,0.71,3.25,0.00,0.00
d,0.83,[],0.71,3.75,0.67,4.50,0.04,-0.75
e,0.78,[],0.63,6.00,0.54,6.50,0.08,-0.50
f,0.72,[],0.58,6.25,0.63,4.75,-0.04,1.50
g,0.67,[],0.63,6.50,0.58,7.00,0.04,-0.50
h,0.61,[],0.54,8.00,0.54,8.00,0.00,0.00
i,0.56,[],0.50,8.25,0.50,9.00,0.00,-0.75
k,0.44,[],0.46,10.50,0.50,10.00,-0.04,0.50
